### Installing required packages

In [ ]:
! pip install numpy
! pip install pandas

In [ ]:
import numpy as np
import pandas as pd

### Data extraction

In [ ]:
def prepare_data(data, variant):
  file = open(f"./data/{variant}.txt")

  while True:
    content=file.readline()
    if not content:
      break
    
    if content[0] == ">":
      headers = content[:-2].split("|")
      name = headers[0].split("/")[-2]
      access_id = headers[1]
      collection_date = headers[2]
      data.append({
        "variant": variant,
        "name": name,
        "access_id": access_id,
        "collection_date": collection_date,
        "sequence": ""
      })
    else:
      data[-1]["sequence"] = data[-1]["sequence"] + content[:-1]
  file.close()

In [ ]:

variants = ["alpha", "beta", "gamma", "delta", "omicron"]
data = []
for variant in variants:
  prepare_data(data, variant)

filtered_data = []
for row in data:
  if not("N" in row["sequence"]):
    filtered_data.append(row)
df = pd.DataFrame(filtered_data)
df.head()